In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

experiments = pd.read_parquet("results_bernoulli_skewed.parquet")


In [2]:
from distributions import SkeGTD
rng = np.random.default_rng(0)
dist = SkeGTD(a=np.inf, r=0.9, rng=rng)
dist2 = SkeGTD(a=1.005, r=0.9, rng=rng)
dist3 = SkeGTD(a=1.005, r=0.95, rng=rng)
dist4 = SkeGTD(a=1.005, r=0.99, rng=rng)
def mean_f(x):
    if x=="Skewed Gaussian":
        return dist.mean()
    elif x=="St r=0.9":
        return dist2.mean()
    elif x=="St r=0.95":
        return dist3.mean()
    elif x=="St r=0.99":
        return dist4.mean()
means = experiments["distribution"].apply(lambda x: mean_f(x))
experiments["error"] = np.abs(experiments["estimate"]-means)

In [3]:
range_ns = np.unique(experiments["n"])
range_deltas = -np.sort(-np.unique(experiments["delta"]))
range_distributions = np.unique(experiments["distribution"])
range_c_etas = np.sort(np.unique(experiments["c_eta"]))

# organize results
df_grouped = [ experiments[ experiments["delta"] == delta ].groupby(["n", "delta", "method", "distribution", "c_eta"])["error"].quantile(1-delta).reset_index() for delta in range_deltas ]
df_grouped = pd.concat(df_grouped)

df_grouped_pivot = df_grouped.pivot(index=['n', 'delta', 'distribution', "c_eta"], columns='method', values='error').reset_index()

relative_errors = df_grouped_pivot.copy()
est_cols = [col for col in df_grouped_pivot.columns if col not in ['n', 'delta', 'distribution', "c_eta"]]
for est in est_cols:
    relative_errors[est] = (df_grouped_pivot[est] -  df_grouped_pivot["mean"])/df_grouped_pivot["mean"]

#df_grouped
relative_errors

method,n,delta,distribution,c_eta,mean,mean atm,mean exp,mean lv,mean win,mom,mom atm,mom exp,mom lv,mom win,tm,tm atm,tm exp,tm lv,tm win
0,100,0.001,Skewed Gaussian,0.5,0.0,0.306339,5.957846e-02,5.957846e-02,0.179487,9.154329e-02,0.328755,5.957846e-02,6.688046e-02,0.184199,0.061216,0.376110,7.453117e-02,6.788203e-02,0.247027
1,100,0.001,Skewed Gaussian,1.0,0.0,0.243623,1.394023e-01,1.396802e-01,0.002083,2.653590e-01,0.308920,1.464054e-01,1.450556e-01,0.093237,0.156961,0.369165,2.197181e-01,2.182785e-01,0.082821
2,100,0.001,Skewed Gaussian,2.0,0.0,0.260527,2.032773e-01,2.983952e-01,0.113973,1.825409e-01,0.409178,2.655265e-01,3.726690e-01,0.220381,0.327348,0.626924,3.771182e-01,5.027729e-01,0.395549
3,100,0.001,St r=0.9,0.5,0.0,-0.334617,-3.344809e-01,-7.963975e-02,-0.705283,-5.905559e-01,-0.696619,-7.308134e-01,-7.182927e-01,-0.733086,-0.734858,-0.687377,-7.197067e-01,-7.052410e-01,-0.727930
4,100,0.001,St r=0.9,1.0,0.0,-0.246382,-1.222318e-01,-4.570116e-03,-0.700729,-6.755852e-01,-0.682687,-7.108285e-01,-6.935756e-01,-0.720050,-0.720999,-0.663876,-6.985362e-01,-6.845884e-01,-0.700793
5,100,0.001,St r=0.9,2.0,0.0,-0.015417,-2.477212e-02,-2.766435e-03,-0.655551,-7.299759e-01,-0.676095,-7.064862e-01,-6.887737e-01,-0.721364,-0.695584,-0.650645,-6.773300e-01,-6.609750e-01,-0.696734
6,100,0.010,Skewed Gaussian,0.5,0.0,0.591021,3.779545e-02,4.000649e-02,0.038471,1.381405e-01,0.621931,3.750952e-02,3.750952e-02,0.063859,0.079611,0.658881,4.867214e-02,4.962816e-02,0.078118
7,100,0.010,Skewed Gaussian,1.0,0.0,0.654738,9.403366e-02,7.981067e-02,0.190647,1.853240e-01,0.716685,9.422510e-02,7.153520e-02,0.215101,0.139933,0.785336,1.309801e-01,9.296948e-02,0.293395
8,100,0.010,Skewed Gaussian,2.0,0.0,0.390369,2.027778e-01,1.484330e-01,0.190483,1.743206e-01,0.557843,2.348108e-01,1.726827e-01,0.267196,0.273366,0.794680,3.362598e-01,2.421956e-01,0.400499
9,100,0.010,St r=0.9,0.5,0.0,-0.338612,-4.108663e-01,-3.714733e-01,-0.438759,-3.227184e-01,-0.341042,-4.071848e-01,-3.740135e-01,-0.434339,-0.444358,-0.328735,-3.878811e-01,-3.506914e-01,-0.422683


In [4]:
experiments_split = pd.read_parquet("results_bernoulli_skewed_split.parquet")
means = experiments_split["distribution"].apply(lambda x: mean_f(x))
experiments_split["error"] = np.abs(experiments_split["estimate"]-means)

range_ns_split = np.unique(experiments_split["n"])
range_deltas_split = -np.sort(-np.unique(experiments_split["delta"]))
range_distributions_split = np.unique(experiments_split["distribution"])
range_c_etas_split = np.sort(np.unique(experiments_split["c_eta"]))

# organize results
df_grouped_split = [ experiments_split[ experiments_split["delta"] == delta ].groupby(["n", "delta", "method", "distribution", "c_eta"])["error"].quantile(1-delta).reset_index() for delta in range_deltas_split ]
df_grouped_split = pd.concat(df_grouped_split)

df_grouped_pivot_split = df_grouped_split.pivot(index=['n', 'delta', 'distribution', "c_eta"], columns='method', values='error').reset_index()

relative_errors_split = df_grouped_pivot_split.copy()
est_cols_split = [col for col in df_grouped_pivot_split.columns if col not in ['n', 'delta', 'distribution', 'c_eta']]
for est in est_cols_split:
    relative_errors_split[est] = (df_grouped_pivot_split[est] -  df_grouped_pivot_split["mean"])/df_grouped_pivot_split["mean"]

assert (range_ns_split==range_ns).all()
assert (range_deltas_split==range_deltas).all()
assert (range_distributions_split==range_distributions).all()
assert est_cols_split==est_cols
assert (range_c_etas == range_c_etas_split).all()


In [5]:
best_dicts = dict()

for delta in range_deltas:
    best_dicts[delta] = dict()
    for n in range_ns:
        best_dicts[delta][n]=dict()
        for distribution in range_distributions:
            best_dicts[delta][n][distribution]=dict()
            for c_eta in range_c_etas:
                filtered = relative_errors[ (relative_errors.n == n) & (relative_errors.delta == delta) & (relative_errors.distribution == distribution) & (relative_errors.c_eta == c_eta)].T[4:]
                best_dicts[delta][n][distribution][c_eta]=list(filtered.iloc[np.argsort(np.array(filtered).flatten())[:2]].index)

name_dict = {
    "mean": "$\\overline{X}$",
    "tm": "TM",
    "mom": "MoM",
    "win": "$1 \\wedge  t^{-1}$",
    "lv": "$(1-t^2)_+$ ",
    "exp": "$e^{-t^2}$ ",
    "atm": "$\mathbf{1}_{t < 1}$"
}

for distribution in range_distributions:
    print()
    for n in range_ns:
        print()
        print("\\begin{table}[t!]")
        print("\t\\centering")
        print("\\setlength\\tabcolsep{3pt}")

        print("\\begin{tabular}{l|cc|cc|cc|cc|cc|cc|cc|cc|cc}")
        print("& \\multicolumn{6}{c|}{\\( \delta = 0.1 \\)} & \multicolumn{6}{c|}{\\( \delta = 0.01 \\)} & \multicolumn{6}{c}{\\( \delta = 0.001 \\)} \\\\")
        print("& \\multicolumn{2}{c}{\\( c_\\eta = 0.5 \\)} & \\multicolumn{2}{c}{\\( c_\\eta = 1 \\)} & \\multicolumn{2}{c|}{\\( c_\\eta = 2 \\)} & \\multicolumn{2}{c}{\\( c_\\eta = 0.5 \\)} & \\multicolumn{2}{c}{\\( c_\\eta = 1 \\)} & \\multicolumn{2}{c|}{\\( c_\\eta = 2 \\)} & \\multicolumn{2}{c}{\\( c_\\eta = 0.5 \\)} & \\multicolumn{2}{c}{\\( c_\\eta = 1 \\)} & \\multicolumn{2}{c}{\\( c_\\eta = 2 \\)} \\\\")
        print("Estimator & \\( \ovoid \\) & \\(\\oslash\\) & \\( \ovoid \\) & \\(\oslash\\) & \\( \ovoid \\) & \\(\\oslash\\) & \\( \\ovoid \\) & \\(\\oslash\\) & \\( \\ovoid \\) & \\(\\oslash\\) & \\( \\ovoid \\) & \\(\\oslash\\) & \\( \\ovoid \\) & \\(\\oslash\\) & \\( \\ovoid \\) & \\(\\oslash\\) & \\( \\ovoid \\) & \\(\\oslash\\) \\\\")

        for est in est_cols:
            if " " in est:
                line = name_dict[est.split(" ")[1]]
                #kappa, rho = est.split(" ")
                #kappa = name_dict[kappa]
                #rho = name_dict[rho]
                #if "$" in kappa:
                #    kappa = kappa.replace("$", "")
                #else:
                #    kappa = f"\\text{{{kappa}}}"
                #rho = rho.replace("$", "")
                #line = f"$\\widehat{{\\kappa}}={kappa}, \\rho(t)={rho}$"
            else:
                line = name_dict[est]
            if " " not in est:
                print("\\hline")
            for delta in range_deltas:
                for c_eta in range_c_etas:
                    best = False
                    if est in best_dicts[delta][n][distribution][c_eta]:
                        best = True
                    if " " in est:
                        change = relative_errors[ (relative_errors.n == n) & (relative_errors.delta == delta) & (relative_errors.delta == delta) & (relative_errors.distribution == distribution) & (relative_errors.c_eta == c_eta)][est].values[0]
                        if best:
                            line += f" & \\(\\mathbf{{{change*100:.0f}}}\\)"
                        else:
                            line += f" & \\({change*100:.0f}\\)"
                        change = relative_errors_split[ (relative_errors_split.n == n) & (relative_errors_split.delta == delta) & (relative_errors_split.delta == delta) & (relative_errors_split.distribution == distribution)  & (relative_errors_split.c_eta == c_eta)][est].values[0]
                        line += f" & \\({change*100:.0f}\\)"
                    else:
                        change = change = relative_errors[ (relative_errors.n == n) & (relative_errors.delta == delta) & (relative_errors.delta == delta) & (relative_errors.distribution == distribution) & (relative_errors.c_eta == c_eta)][est].values[0]
                        change_split = relative_errors_split[ (relative_errors_split.n == n) & (relative_errors_split.delta == delta) & (relative_errors_split.delta == delta) & (relative_errors_split.distribution == distribution)  & (relative_errors_split.c_eta == c_eta)][est].values[0]
                        assert change == change_split
                        if c_eta == range_c_etas[-1] and delta == range_deltas[-1]:
                            if best:
                                line += f" & \\multicolumn{{2}}{{c}}{{\\(\\mathbf{{{change*100:.0f}}}\\)}}"  
                            else:
                                line += f" & \\multicolumn{{2}}{{c}}{{\\({change*100:.0f}\\)}}"
                        else:
                            if best:
                                line += f" & \\multicolumn{{2}}{{c|}}{{\\(\\mathbf{{{change*100:.0f}}}\\)}}"  
                            else:
                                line += f" & \\multicolumn{{2}}{{c|}}{{\\({change*100:.0f}\\)}}"
            line += "\\\\"
            print(line)
        print("\\hline")
        print("\\end{tabular}")
        print(f"\\caption{{Relative difference (in \\%) of the empirical \\( 1-\\delta  \\) confidence interval error of the indicated estimators with respect to that of the empirical mean under a {distribution} with \\(n={n}\\).}}")
        print(f"\\label{{tab:experiment{distribution}_n={n}}}")
        print("\\end{table}")




\begin{table}[t!]
	\centering
\setlength\tabcolsep{3pt}
\begin{tabular}{l|cc|cc|cc|cc|cc|cc|cc|cc|cc}
& \multicolumn{6}{c|}{\( \delta = 0.1 \)} & \multicolumn{6}{c|}{\( \delta = 0.01 \)} & \multicolumn{6}{c}{\( \delta = 0.001 \)} \\
& \multicolumn{2}{c}{\( c_\eta = 0.5 \)} & \multicolumn{2}{c}{\( c_\eta = 1 \)} & \multicolumn{2}{c|}{\( c_\eta = 2 \)} & \multicolumn{2}{c}{\( c_\eta = 0.5 \)} & \multicolumn{2}{c}{\( c_\eta = 1 \)} & \multicolumn{2}{c|}{\( c_\eta = 2 \)} & \multicolumn{2}{c}{\( c_\eta = 0.5 \)} & \multicolumn{2}{c}{\( c_\eta = 1 \)} & \multicolumn{2}{c}{\( c_\eta = 2 \)} \\
Estimator & \( \ovoid \) & \(\oslash\) & \( \ovoid \) & \(\oslash\) & \( \ovoid \) & \(\oslash\) & \( \ovoid \) & \(\oslash\) & \( \ovoid \) & \(\oslash\) & \( \ovoid \) & \(\oslash\) & \( \ovoid \) & \(\oslash\) & \( \ovoid \) & \(\oslash\) & \( \ovoid \) & \(\oslash\) \\
\hline
$\overline{X}$ & \multicolumn{2}{c|}{\(\mathbf{0}\)} & \multicolumn{2}{c|}{\(\mathbf{0}\)} & \multicolumn{2}{c|}{\(\mathbf

In [22]:
experiments["error"].mean()

np.float64(1.726859503477982)

In [5]:
experiments

,n,method,estimate,delta,distribution,error
0,50,tm,-0.022962,0.100,Gaussian,0.022962
1,50,tm,-0.073220,0.100,Gaussian,0.073220
2,50,tm,-0.117201,0.100,Gaussian,0.117201
3,50,tm,-0.056495,0.100,Gaussian,0.056495
4,50,tm,-0.237023,0.100,Gaussian,0.237023
...,...,...,...,...,...,...
1499995,500,mom exp,-0.021399,0.001,St df=2.01,0.021399
1499996,500,mom exp,0.152496,0.001,St df=2.01,0.152496
1499997,500,mom exp,0.045685,0.001,St df=2.01,0.045685
1499998,500,mom exp,-0.095915,0.001,St df=2.01,0.095915


In [8]:
experiments[ experiments["delta"] == delta ].groupby(["n", "delta", "method", "distribution", "c_eta"])["error"]

In [14]:
len(experiments[ (experiments["delta"] == delta) & (experiments["n"] == n) & (experiments["distribution"] == distribution) & 
    (experiments["method"] == "tm") & (experiments["c_eta"] == c_eta)])


10000